In [ ]:
import numpy as np
import pandas as pd
import sklearn as sk
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import display
import seaborn as sn

print('numpy version:', np.__version__)
print('pandas version:', pd.__version__)
print('scikit-learn version:', sk.__version__)
print('matplotlib version:', matplotlib.__version__)

%matplotlib inline

In [ ]:
dfRelevant = pd.DataFrame()

In [ ]:
df = pd.read_csv("../data/IGN_data.csv",index_col=0)


In [ ]:
df.sample(10)

In [ ]:
df.info()

here I take out all the null values of the values I think is important

In [ ]:
dfPub = df
dfPub = dfPub[dfPub.publishers.notna()]
dfDates = dfPub
dfDates = dfPub[dfPub.released_date.notna()]
dfGenres = dfDates
dfGenres = dfDates[dfDates.genres.notna()]
dfFeatures = dfGenres
dfFeatures = dfFeatures[dfFeatures.features.notna()]
dfScore = dfFeatures
dfScore = dfScore[dfScore.score.notna()]
dfScore.info()



In [ ]:
shape = dfScore.shape


dfRelevant["released_date_f"] = pd.to_datetime(dfScore["released_date"]).astype(int) / 10**9
# creating a column with datetime properly formatted

# creating predictors to build basic ML model that we can add further complexity to later

# dfGenres["publishers_f"] =dfGenres["publishers"].astype("category").cat.codes # ->one hot encoding
# dfGenres["features_f"] = dfGenres["features"].astype("category").cat.codes # ->one hot encoding
# dfGenres["platform_f"] = dfGenres["platform"].astype("category").cat.codes # ->one hot encoding
# dfGenres["genres_f"] = dfGenres["genres"].astype("category").cat.codes # ->one hot encoding
dfRelevant["esrb_f"] = dfScore["esrb"].astype("category").cat.codes
dfRelevant["score_text_f"] = dfGenres["score_text"].astype("category").cat.codes
dfRelevant["franchises_f"] = dfScore["franchises"].astype("category").cat.codes
#converting from string to categories and then converting those categoties to numbers

## One hot encoding

In [ ]:
dfHotFeatures = pd.DataFrame()
dfHotFeatures['score'] = dfScore['score']

In [ ]:
for row in dfScore.features:
         if (row):
                row_ = row.strip('[]')
                row_ = row.replace("'", "")
                rowArr = row_.split(',')
                for feature in rowArr:
                    feature = feature.replace('[', "")
                    feature = feature.replace(']', "")
                    dfRelevant.loc[dfFeatures['features'] == row, feature] = 1
                    dfHotFeatures.loc[dfFeatures['features'] == row, feature] = 1
                    


Here I show the relation between the score and hot encoded features

In [ ]:
print(dfHotFeatures.shape)
dfHotFeatures = dfHotFeatures.fillna(0)
columns = dfRelevant.shape[1]

dfTemp = dfHotFeatures
corr_matrix = dfTemp.corr()
sn.heatmap(corr_matrix.iloc[[0]], annot=False)
plt.show()


In [ ]:
dfHotPublishers = pd.DataFrame()
dfHotPublishers['score'] = dfScore['score']

In [ ]:
for row in dfScore.publishers:
         if (row):
                row_ = row.replace('[', "")
                row_ = row.replace(']', "")
                row_ = row.replace("'", "")
                rowArr = row_.split(',')
                for publisher in rowArr:
                    publisher = publisher.replace('[', "")
                    publisher = publisher.replace(']', "")
                    dfRelevant.loc[dfFeatures['publishers'] == row, publisher] = 1
                    dfHotPublishers.loc[dfFeatures['publishers'] == row, publisher] = 1


In [ ]:
print(dfHotPublishers.shape)
dfHotPublishers = dfHotPublishers.fillna(0)
dfTemp = dfHotPublishers
corr_matrix = dfTemp.corr()
sn.heatmap(corr_matrix.iloc[[0]], annot=False)
plt.show()


In [ ]:
dfHotPlatform = pd.DataFrame()
dfHotPlatform['score'] = dfScore['score']

In [ ]:
for row in dfScore.platform:
         if (row):
                row_ = row.replace('[', "")
                row_ = row.replace(']', "")
                row_ = row.replace("'", "")
                rowArr = row_.split(',')
                for platform in rowArr:
                    platform = platform.replace('[', "")
                    platform = platform.replace(']', "")
                    dfRelevant.loc[dfFeatures['platform'] == row, platform] = 1
                    dfHotPlatform.loc[dfFeatures['platform'] == row, platform] = 1


In [ ]:
print(dfHotPlatform.shape)
dfHotPlatform = dfHotPlatform.fillna(0)
dfTemp = dfHotPlatform
corr_matrix = dfTemp.corr()
sn.heatmap(corr_matrix.iloc[[0]], annot=False)
plt.show()

In [ ]:
for row in dfScore.genres:
         if (row):
                row_ = row.replace('[', "")
                row_ = row.replace(']', "")
                row_ = row.replace("'", "")
                rowArr = row_.split(',')
                for genre_ in rowArr:
                    genre_ = genre_.replace('[', "")
                    genre_ = genre_.replace(']', "")
                    dfRelevant.loc[dfFeatures['genres'] == row, genre_] = 1


In [ ]:
dfRelevant

In [ ]:
dfRelevant.info()
dfRelevant = dfRelevant.fillna(0)

In [ ]:
corr_matrix = corr_matrix.replace(to_replace=1, value=-1)
corr_matrix = corr_matrix.reset_index(drop=True)
# inloop = 0
# for columns in corr_matrix.items():
#    if(inloop):
#    # print(columns)
#       print(max(columns))
#    inloop=1

In [ ]:
from sklearn.model_selection import train_test_split
scoring_X = dfRelevant
scoring_y = (dfScore['score']*10).astype(int)

X_train_scoring, X_test_scoring, y_train_scoring, y_test_scoring = train_test_split(scoring_X, scoring_y, test_size=0.2)

In [ ]:
X_test_scoring

In [ ]:
y_test_scoring

## data visualisation

In [ ]:
# dfTemp = X_test_scoring
# dfTemp["score"] = y_test_scoring # for some reason this adds the score line to the X_test_scoring dataframe as well???????

# ax1 = dfTemp.plot.scatter(x='released_date_f',y='score',c='esrb_f')
# ax2 = dfTemp.plot.scatter(x='esrb_f',y='score')
# ax3 = dfTemp.plot.scatter(x='esrb_f',y='franchises_f',c='score')
# ax4 = dfTemp.plot.scatter(x='franchises_f',y='released_date_f',c='score')

# ax5 = dfTemp.plot.scatter(x='released_date_f',y='score')


### Relevancy
these visualisations don't show a clear correlation between any of the prominent features. I hope maybe some of the other features like the genres or platforms have a significant influence on the score, but I highly doubt those will show any correlation either. I will explore this below in a correlation matrix.

In [ ]:
plt.boxplot(dfTemp['score'])
plt.show()

### correlation matrix

here I try to look for correlation in the data to see what is relevant for the AI.

Because of the sheer amount of columns I have (almost 2000) I will split this up into several matrices or the kernel will crash because of too much data.

## DecisionTree and RandomForest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier


treeModel = DecisionTreeClassifier(criterion='entropy', max_depth=5)
treeModel.fit(X_train_scoring,y_train_scoring)
tree.plot_tree(treeModel)

y_pred_tree = treeModel.predict(X_test_scoring)
accuracy_score(y_test_scoring, y_pred_tree)

In [ ]:
rf = RandomForestClassifier(n_estimators=50, min_samples_split=2)
rf.fit(X_train_scoring,y_train_scoring)

y_pred = rf.predict(X_test_scoring)
accuracy_score(y_test_scoring, y_pred)

## kNN